In [1]:
import requests

PORT = "9200"
URL = "http://localhost:" + PORT + "/"

def parse_analysis(string):
    split_input = [x.strip() for x in string.split("\t") if x != ""]
    
    if len(split_input) != 3:
        return ()
    
    base, form_explicit, _disamb = split_input
    form = form_explicit.split(":")[0]
    
    return (base, form)

def analyze(string):
    content = requests.post(URL, data=string.encode('utf-8')).text
    analyzed_words = [parse_analysis(analysis) for analysis in content.split("\n")[1::2] if analysis != ""]
    
    return [analysis for analysis in analyzed_words if analysis != ()]
    
def is_valid_unigram(unigram):
    word, analysis = unigram
    return analysis != "interp" and not analysis.startswith("num")


In [4]:
analyze("ala ma kota\n\n\n\n12     123.")

[('Ala', 'subst'), ('mieć', 'fin'), ('kot', 'subst')]

In [5]:
FILE_LIST = 'files.p'
DATA_DIR = "/run/media/maciej/Nowy/data/json/"
CHOSEN_YEAR = "2011"

In [6]:
import regex as re

judgment_types = {
    "I":  "A?C.*",
    "II": "A?U.*",
    "III": "A?K.*",
    "IV": "G.*",
    "V": "A?P.*",
    "VI": "R.*",
    "VII": "W.*",
    "VIII": "Am.*",
}

def get_judgment_type(number):
    for key, value in judgment_types.items():
        if re.search(value, number):
            print("Found a match for {}".format(number))
            return key
    print("Did not match for {}".format(
        number
    ))

In [17]:
import os, json, pickle
from collections import defaultdict

import regex
from tqdm import tqdm

word_pattern = "\p{Letter}+"


def load_data():
    total_judgments = []
    files = pickle.load(open(FILE_LIST, 'rb'))
    
    for file in tqdm(files):
        if file.startswith("judgment"):
            file_path = os.path.join(DATA_DIR, file)

            with open(file_path, 'r') as f:
                data = json.load(f)
                judgments = [
                    (x["textContent"], x["courtCases"][0]["caseNumber"]) for x in data["items"] 
                    if x["judgmentDate"].startswith(CHOSEN_YEAR) and x["courtType"] in ("COMMON", "SUPREME")
                ]
                
            total_judgments += judgments
    
    print("Found {} judgments from common and supreme courts".format(len(total_judgments)))
    
    judgments_with_explanations = []    
    for judgment, _case_num in total_judgments:        
        match = regex.search("uzasadnienie:", judgment, regex.IGNORECASE)        
        if match:
            print(judgment)
            break
            judgments_with_explanations.append((judgment[match.end():], _case_num))
            
    print("{} of them have explanation section".format(len(judgments_with_explanations)))
    
    analyzed_judgments = defaultdict(lambda: [])
    raw_judgments = defaultdict(lambda: [])
    
#     for judgment, case_number in tqdm(judgments_with_explanations):
        
#         judgment = regex.sub("<.*?>", "", judgment)
#         judgment = regex.sub("-\n(\p{Letter}+)", r"\1", judgment)
#         judgment = judgment.lower()

#         judgment_type = get_judgment_type(case_number)

#         raw_judgments[judgment_type].append(judgment)
#         analyzed_judgments[judgment_type].append(analyze(judgment))
        
    
    return analyzed_judgments, raw_judgments

analyzed_judgments, raw_judgments = load_data()

100%|██████████| 68/68 [00:01<00:00, 53.34it/s]

Found 3791 judgments from common and supreme courts
Sygn. akt III KRS 4/11
POSTANOWIENIE
Dnia 11 maja 2011 r.
Sąd Najwyższy w składzie :
SSN Kazimierz Jaśkowski (przewodniczący)
SSN Zbigniew Hajn
SSN Andrzej Wróbel (sprawozdawca)
Protokolant Małgorzata Beczek
w sprawie z odwołania Kolegium Sądu Okręgowego
od uchwały Krajowej Rady Sądownictwa z dnia 17 listopada 2010 r. w sprawie
odmowy przeniesienia sędziego Sądu Rejonowego /.../ Z. W. w stan spoczynku,
po rozpoznaniu na rozprawie w Izbie Pracy, Ubezpieczeń Społecznych i Spraw
Publicznych w dniu 11 maja 2011 r.,
odrzuca odwołanie.
Uzasadnienie:
Kolegium Sądu Okręgowego, działając na podstawie art. 73 § 2 i 3 Prawa o
ustroju sądów powszechnych w związku z art. 13 ust. 2 ustawy z dnia 27 lipca 2001
r. o Krajowej Radzie Sądownictwa (jednolity tekst: Dz.U. z 2010 r. Nr 11, poz. 67)
zaskarżyło uchwałę Krajowej Rady Sądownictwa z dnia 17 listopada 2010 r. nr
1922/2010 w sprawie odmowy przeniesienia w stan spoczynku sędziego Sądu
Rejonowego Z